# dependencias

In [1]:
import igraph as ig
import json

# lectura de los datos 

In [2]:
# leemos el json
with open('../input/graph/Universidad Nacional de Colombia-Bogotá-FACULTAD DE CIENCIAS-Departamento de Estadística.json') as f:
    data = json.load(f)

In [ ]:
data['']

In [3]:
# nodos 
nodes = data['nodes']
# aristas
edges = data['links']
# info aparte 
options = data['options']

In [10]:
nodes[0]

{'id': '2d4d5a17-7ef0-4fc4-8b25-d1af93445f50',
 'label': 'Departamento de Estadística',
 'type': 'uab'}

In [12]:
nodos = [nodo['id'] for nodo in nodes if nodo['id'] != '2d4d5a17-7ef0-4fc4-8b25-d1af93445f50']
len(nodos)

363